In [11]:
%matplotlib inline
import numpy as np
import scipy.integrate as integ
import matplotlib.pyplot as plt

def f(x,y,a,b):
    return np.exp(-(((x-a)/a)**2+((y-b)/b)**2)/2)/(a*b*2*np.pi)
#y=np.linspace(-100,100,10000)
#plt.plot(y,integ.quad(f, -np.inf, np.inf,args=(3,4,y)))
#x,y=np.meshgrid(np.linspace(-10,10,100),np.linspace(-10,10,100))
#plt.contourf(x,y,f(x,y,10,4))
#plt.show()
#res,err=integ.quad(f, -np.inf, np.inf,args=(0,3,4))
#print(res*np.sqrt(2*np.pi)*4*np.exp(1/2))
#print(err)
def f2(x,a):
    return np.exp(-(x/a)**2/2)/(a*np.sqrt(2*np.pi) )
res,err=integ.quad(f2, -100, 100,args=(1,))
#plt.figure(2)
#plt.plot(np.linspace(-20,20,100),f2(np.linspace(-20,20,100),1))
#plt.show()
print(res)
print(err)
def fint(a,b):
    return integ.dblquad(f, -np.inf, np.inf, lambda x,a,b: 0, lambda x,a,b: 1, args=(a,b))
    #return integ.dblquad(f, -np.inf, np.inf, lambda x: 0, lambda x: np.inf,args=(a,b))
    #return integ.quad(lambda a,b,y: integ.quad(f, -np.inf, np.inf,args=(a,b,y)),-np.inf,np.inf,args=(a,b))
    #return integ.quad(f, -np.inf, np.inf,args=(a,b,0))
res,err = fint(3,4)
print(res)
print(err)
#def f(a,b,x):
#    return np.exp(-(((x-a)/b)**2)/2)/(b*np.sqrt(2*np.pi))
#def fint(a,b):
#    return integ.quad(f, -np.inf, np.inf,args=(a,b))
#res,err = fint(3,4)
#res

1.0000000000000002
1.0346447361664605e-12


TypeError: <lambda>() takes 1 positional argument but 3 were given

In [6]:
import sympy as sym

a,b,x,y = sym.symbols(" a b x y")
def fsym(a,b,x,y):
    return sym.exp(-(((x-a)/a)**2+((y-b)/b)**2)/2)/(a*b*2*sym.pi)
sym.N(sym.integrate(fsym(3,4,x,0),(x,-sym.oo, sym.oo))*sym.sqrt(sym.pi*2)*4*sym.exp(1/2))

1.00000000000000